Here are the functions used during data preprocessing, training, and testing:

In [ ]:

##Use the tokenizer to encode the text
def encode(data_component):
    encoded_data = tokenizer(data_component, return_tensors='pt', padding="max_length", truncation=True, max_length=512)  #Force max length here to make sure tensors fit
    return encoded_data["input_ids"], encoded_data["attention_mask"]


##outputs the accuracy of the model
def accuracy(predictions, labels):
    preds = torch.argmax(torch.softmax(predictions, dim=1), dim=1)  # Get class with highest probability
    return (preds == labels).float().mean().item()



##Training Loop
def train(bert_classifier, device, trainloader, valloader, criterion, optimizer, epochs):
    running_acc = []  # Store accuracy values for logging

    for epoch in range(epochs):
        epoch_loss = 0.0  
        training_acc = 0.0  
        training_loss = 0.0
        
        bert_classifier.train()  # Set model to training mode

        # Training loop
        for i, batch in enumerate(trainloader):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device).long()  
            
            optimizer.zero_grad()  # Zero the gradients
            
            # Forward pass
            logits = bert_classifier(input_ids=input_ids, attention_mask=attention_mask)
            
            # Compute loss
            loss = criterion(logits, labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()  # Update weights
            
            # Accumulate loss and accuracy
            epoch_loss += loss.item()
            training_loss += loss.item()
            training_acc += accuracy(logits, labels)  # Assume `accuracy` function is defined

            # Print loss and accuracy every 50 batches
            if (i + 1) % 50 == 0: 
                avg_acc = training_acc / 50  # Normalize by batch count
                avg_loss = training_loss / 50

                print(f'Epoch {epoch}, Batch {i+1}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')
                
                running_acc.append(avg_acc)

                # Log accuracy without visualization
                running_acc_df = pd.DataFrame({'Batch': range(len(running_acc)), 'Accuracy': running_acc})
                print(running_acc_df.tail(5))  # Print last 5 rows instead of plotting
                
                # Reset batch accumulators
                training_loss = 0.0
                training_acc = 0.0

        # Validation loop
        running_vloss = 0.0
        bert_classifier.eval()  # Set model to evaluation mode

        with torch.no_grad():
            val_acc = 0.0
            val_steps = 0
            
            for vbatch in valloader:
                vinputs = vbatch[0].to(device)
                vattention_mask = vbatch[1].to(device)
                vlabels = vbatch[2].to(device).long()

                vlogits = bert_classifier(input_ids=vinputs, attention_mask=vattention_mask)
                vloss = criterion(vlogits, vlabels)
                
                running_vloss += vloss.item()
                val_acc += accuracy(vlogits, vlabels)  
                val_steps += 1

            avg_vloss = running_vloss / val_steps
            avg_vacc = val_acc / val_steps

            print(f'Epoch {epoch}, Training Loss: {epoch_loss / len(trainloader):.4f}, '
                  f'Validation Loss: {avg_vloss:.4f}, Validation Accuracy: {avg_vacc:.4f}')


#Testing Loop        
def test(model, device, testloader, criterion, epoch):
    # Set the model to evaluation mode
    bert_classifier.eval()
    TestAccuracy = []
    correct_predictions = []
    
    
    for epoch in range(epochs):
        epoch_loss = 0.0  # To accumulate loss for this epoch
        testing_acc = 0.0  # Accumulate accuracy
        testing_loss = 0.0
        
        

    #loop over the batches of data:
        with torch.no_grad():
            for i, batch in enumerate(testloader):
                input_ids = batch[0].to(device)
                attention_mask = batch[1].to(device)
                labels = batch[2].to(device)
                inputs = {
                    'input_ids': input_ids,
                    'attention_mask': attention_mask
                }
                
                
                #forward pass
                logits = model(inputs)
            
                
                #calculate the loss
                loss = criterion(logits, labels) # Calculate the loss
               
                
                # Accumulate loss for this batch
                epoch_loss += loss.item()
                testing_loss += loss.item()
                testing_acc+= accuracy(logits, labels)

                                
                if (i + 1) % 200 == 0: # print every two hundred batches
                    print(f'Epoch {epoch}, Batch {i+1}, Loss: {testing_loss / 200:.4f}, Accuracy: {testing_acc / 200:.4f}')

Import all the necessary libraries:

In [ ]:
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split 
from transformers.models.bert.modeling_bert import BertIntermediate, BertOutput, BertEncoder, BertSelfAttention, BertSelfOutput, BertModel, BertConfig

Import the question sets that will be used to train the model.  The first dataset used was the RACE dataseet, which consists of multiple choice questions separated between M (middle school) and H (high school).  The second dataset is QxGrade_dataset which is a set of 75k questions scraped from pdf textbooks.  These textbooks were chosen based on alignment with Common Core State Standards to identify a framework that we can use when training the model with additional data. 

In [ ]:
df = pd.read_csv('QxGrade_Dataset.csv')

In [ ]:
df.columns

The two most important columns we will be using and labeling are Grade and Question.  Using the .values and .tolist function here we are adding all of the grade options (3-12) to the grades function.  We are doing the same with all of the question values.

In [ ]:
questions = df.question.values.tolist()
grades = df.Grade.astype(str).tolist()

Tokenize the data using the encode function

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
q_tensors, attention_mask_q = encode(questions)

In [ ]:
g_tensors, attention_mask_g = encode(grades)

Now set the grade values to 0

In [ ]:
glabels = torch.zeros(len(g_tensors), dtype=torch.float32)

Set the question values to 1

In [ ]:
qlabels = torch.ones(len(q_tensors), dtype=torch.float32)

Concatenate the two

In [ ]:
training_data = torch.cat([g_tensors, q_tensors], dim=1)

Change BERT configuration to implement Dual Multihead Attention Mechanism.  This is also where we are implementing our customized neural network. 

In [ ]:
##This implements a copy of the original attention mechnism to run simultaneously, then at the end, the outputs are joined together
class DualBertAttention(nn.Module):
    
    
    def __init__(self, config):
        super().__init__()
        
        self.attention1 = BertSelfAttention(config)
        self.attention2 = BertSelfAttention(config)
        
        self.output1 = BertSelfOutput(config)
        self.output2 = BertSelfOutput(config)
        
        
    def forward(self, hidden_states, attention_mask=None, head_mask=None):
        attn_output1 = self.attention1(hidden_states, attention_mask, head_mask)[0]  # Unpacking tuple
        attn_output1 = self.output1(attn_output1, hidden_states)

        attn_output2 = self.attention2(hidden_states, attention_mask, head_mask)[0]  # Unpacking tuple
        attn_output2 = self.output2(attn_output2, hidden_states)

        dual_attention_output = F.relu(attn_output1 + attn_output2)
        return dual_attention_output
    
##Implements the dual attention in the Neural Network
class DualBertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = DualBertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)


##Moves the data through the Neural Network
    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        attention_output = self.attention(hidden_states, attention_mask, head_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return (layer_output,)


##Implments the outcome from the DualBertLayer to encode the data from the DualBertLayer Class
class DualBertEncoder(BertEncoder):
    def __init__(self, config):
        super().__init__(config)
        self.layer = nn.ModuleList([DualBertLayer(config) for _ in range(config.num_hidden_layers)])


##Implements the model with the DualBertEncoder
class DualBertModel(BertModel):
    def __init__(self, config):
        super().__init__(config)
        self.encoder = DualBertEncoder(config)


##Initialize a classifier between the 10 different options (3rd grade to 12th grade).

class BertClassifier(nn.Module):
    def __init__(self, bert, hidden_dim=768, num_classes=10): 
        super(BertClassifier, self).__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_dim, num_classes) 

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # CLS token representation
        return self.fc(pooled_output)

Using the network we create, instantiate our version of bert

In [ ]:
config = BertConfig()  # This needs to be an instantiated object, not a class reference
bert = DualBertModel(config)  # Directly create an instance instead of calling from_pretrained


Set up the training hyperparameters

In [ ]:
epochs = 5  ##How many times we go through the loop
criterion = nn.CrossEntropyLoss()  ##This compares the predicted answer with the correct answer
optimizer = optim.Adam(bert.parameters(), lr=1e-5, betas=(0.9, 0.999), weight_decay=1e-5)  ##The Optim.Adam optimizer calculates gradient descent
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  ##Set the device to GPU so we can train the model on the GPU

sequence_length = 156   ## Maximum length of tokens to be used at a time
batch_size = 16  ##The number of training examples in one forward/backward pass
input_dim = 500  ##The total number of dimension we will allow the model to use for calculation
d_model = 512  ##Number of expected features, set to default recommended by pytorch
running_acc = []  ##This is to agregate the training data

Wrap the data in a Dataloader:

In [ ]:
from torch.utils.data.dataloader import default_collate
q_dataset = TensorDataset(q_tensors, attention_mask_q, qlabels.long())  # Convert qlabels to long
g_dataset = TensorDataset(g_tensors, attention_mask_g, glabels.long())  # Convert glabels to l

dataset = torch.utils.data.ConcatDataset([q_dataset, g_dataset])


#split datasets with random split
train_size = int(0.7 * len(dataset))  # 70% training
val_size = int(0.15 * len(dataset))   # 15% validation
test_size = len(dataset) - train_size - val_size  # Remaining 15% test

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])


##Reference torch.utils.data 
def custom_collate(batch):
    inputs = torch.stack([b[0] for b in batch])  # Extract input tensors
    attention_masks = torch.stack([b[1] for b in batch])  # Extract attention masks
    labels = torch.stack([b[2] for b in batch])
    # Default case: use PyTorch's default_collate
    return inputs, attention_masks, labels 


trainloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate)

valloader = DataLoader(val_data, batch_size=32, shuffle=False, collate_fn=custom_collate)  

testloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate )

Move the model to the GPU

In [ ]:
bert.to(device)

Move an instance of the bert classifier to the device as well

In [ ]:
bert_classifier = BertClassifier(bert).to(device)


Use Bert to classify the data

In [ ]:
train(bert_classifier, device, trainloader, valloader, criterion, optimizer, epochs)


Save the model once trained 

In [ ]:
torch.save(bert_classifier.state_dict(), 'Bert_Classifier.pt')